# Connecting to your data

In [ ]:
import datajoint as dj
dj.config['enable_python_native_blobs'] = True
dj.__version__

## Open connection to a database server instance i.e. Pipeline

In [ ]:
# dj.conn(host='db', user='root', password='coogs')
dj.conn()

In [ ]:
dj.list_schemas()

# Structuring your data using a database i.e. Schema

In [ ]:
schema = dj.Schema(f"{dj.config['database.user']}_uh_showcase")

In [ ]:
dj.Di(schema)

In [ ]:
@schema
class Sport(dj.Manual):
    definition = """
    sport_id        :  int auto_increment
    ---
    name      : varchar(30)
    sex="men" : enum("women", "men")
    unique index(name, sex)
    """

In [ ]:
dj.Di(schema)

In [ ]:
Sport.insert([
    dict(name='swim', sex='women'),
    dict(name='soc', sex='women')
])
Sport.insert([
    dict(name='track'),
    dict(name='football')
])

In [ ]:
Sport()

In [ ]:
import datetime

In [ ]:
@schema
class NewsArticle(dj.Manual):
    definition = """
    -> Sport
    news_id       : int
    ---
    date=null: date
    """

In [ ]:
dj.Di(schema)

In [ ]:
NewsArticle.insert([
    [4, 1, datetime.date(2020,3,3)],
    [2, 1, datetime.date(2019,11,5)],
    [1, 1, datetime.date(2020,3,11)],
    [3, 1, datetime.date(2020,3,4)],
])
# NewsArticle.insert([
#     dict(sport_id=1, news_id=1),
#     dict(sport_id=3, news_id=1)
# ])

In [ ]:
NewsArticle()

In [ ]:
import requests
import json
import re
from os import path, mkdir

In [ ]:
@schema
class Headline(dj.Imported):
    definition = """
    -> NewsArticle
    ---
    name : varchar(200)
    image: longblob
    """
    def make(self, key):    
        sport_name, sport_sex = (Sport & key).fetch1('name', 'sex')
        news_date = (NewsArticle & key).fetch1('date')
        file_path = 'cached/{}_{}_{}.JPG'.format(sport_name, sport_sex, news_date)
        
        if not path.exists(file_path):
            base_url = "https://uhcougars.com"
            headers = {
                'User-Agent': "DataJoint"
            }
            querystring = {
                "index":"1",
                "page_size":"30",
                "sport": sport_name if sport_sex != 'women' else sport_sex[0] + sport_name,
                "season":"0"
            }
            response = requests.request("GET", base_url + "/services/archives.ashx/stories", headers=headers, params=querystring)
            article = [v for v in json.loads(response.text)['data'] if v['story_postdate'] == news_date.strftime("%-m/%-d/%Y")][0] if news_date else json.loads(response.text)['data'][0]
            news_date = datetime.datetime.strptime(article['story_postdate'], '%m/%d/%Y').date() if not news_date else news_date
            response = requests.request("GET", base_url + article['story_path'], headers=headers)
            image_path = re.findall(r'<img src="/images/{year}[A-Za-z0-9/_.\-]+'.format(year=news_date.year), response.text)[0][10:]
            response = requests.request("GET", base_url + image_path, headers=headers)
            if not path.exists(path.dirname(file_path)):
                mkdir(path.dirname(file_path))
            with open(file_path, 'wb') as f:
                f.write(response.content)
            print('Image `{}` downloaded.'.format(file_path))
            key['name'] = article['story_headline']
        else:
            key['name'] = 'No Title (loaded from cache)'
            print('Image `{}` read from cache.'.format(file_path))
            
        with open(file_path, mode='rb') as f:
            key['image'] = f.read()
        self.insert1(key)

In [ ]:
dj.Di(schema)

In [ ]:
Headline.populate()

In [ ]:
q_original = Headline()
q_original = Headline & dict(sport_id=1, news_id=1)
q_original

In [ ]:
from PIL import Image
from io import BytesIO
from matplotlib import pyplot as plt

In [ ]:
image_original = q_original.fetch1('image')
image_original = Image.open(BytesIO(image_original))
fig, axarr = plt.subplots(1,1,figsize=(15,15))
axarr.imshow(image_original)

In [ ]:
@schema
class PaintingStyle(dj.Lookup):
    definition = """
    style_name      : varchar(30)
    """
    contents = [
        ['udnie']
    ]

In [ ]:
dj.Di(schema)

In [ ]:
PaintingStyle.insert([
    dict(style_name='la_muse')
])

In [ ]:
PaintingStyle()

In [ ]:
import numpy as np
import cv2
import imutils

In [ ]:
@schema
class Flyer(dj.Computed):
    definition = """
    -> Headline
    -> PaintingStyle
    ---
    image: longblob
    """
    def make(self, key):
        style_path = 'models/' + key['style_name'] + '.t7'
        image = (Headline & key).fetch1('image')
        
        net = cv2.dnn.readNetFromTorch(style_path)
        image = np.frombuffer(image, np.uint8)
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
#         image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
        image = imutils.resize(image, width=600)
        (h, w) = image.shape[:2]

        # construct a blob from the image, set the input, and then perform a
        # forward pass of the network
        blob = cv2.dnn.blobFromImage(image, 1.0, (w, h),
            (103.939, 116.779, 123.680), swapRB=False, crop=False)
        net.setInput(blob)
        output = net.forward()

        # reshape the output tensor, add back in the mean subtraction, and
        # then swap the channel ordering
        output = output.reshape((3, output.shape[2], output.shape[3]))
        output[0] += 103.939
        output[1] += 116.779
        output[2] += 123.680
        output = output.transpose(1, 2, 0)
        output = np.clip(output, 0, 255)
        output= output.astype('uint8')
        
        print('sport_id: {sport_id}, news_id: {news_id}, style_name: {style_name}'.format(**key))
        
        key['image'] = cv2.imencode('.jpg', output)[1].tostring()
        self.insert1(key)

In [ ]:
dj.Di(schema)

In [ ]:
Flyer.populate()

In [ ]:
q_styled = Flyer()
q_styled = Flyer & dict(sport_id=1, news_id=1, style_name='la_muse')
# q_styled = Flyer & dict(sport_id=1, news_id=1, style_name='udnie')
q_styled

In [ ]:
image_styled = q_styled.fetch1('image')
image_styled = Image.open(BytesIO(image_styled))

fig, axarr = plt.subplots(1,2,figsize=(15,15))
axarr[0].imshow(image_original)
axarr[1].imshow(image_styled)

# Clean up and remove generated data

In [ ]:
schema.drop()

In [ ]:
import shutil
shutil.rmtree('cached')